<div class="report-header"><div class="aictx-logo"></div>
<span class="report-type">Demonstration</span><br />
<span class="report-author">Author: Felix Bauer</span><br />
<span class="report-date">25th January, 2020</span>
</div><h1>Live Demo:</h1><h1>ECG anomaly detection</h1>

This notebook demonstrates how recurrent SNNs can be used for anomaly detection in an electrocardiogram (ECG) signal. The main part of the network will run on a DYNAP-SE neuromorphic processor.

In [13]:
%matplotlib widget

### --- Imports
import numpy as np
from matplotlib import pyplot as plt

from dataloader import ECGDataLoader
from rockpool.layers import (
    FFUpDown,
    FFIAFSpkInRefrTorch,
    RecIAFSpkInNest,
    RecDynapSE,
    FFExpSyn,
)
from rockpool import Network

### --- Constants
DT_ECG = 0.002778
NUM_ECG_LEADS = 2
NUM_ECG_CLASSES = 6
MAX_FANIN = 64  # Max. number of presynaptic connections per neuron (hardware limit)

# Task

Our goal is to detect five different classes of anomalies in a two-lead ECG signal from the MIT-BIH Arrithmia Database [ref]. An SNN that fulfills this task can, for instance, be used in a wearable ECG monitoring device to trigger an alarm in presence of pathological patterns. Below we see examples for a normal ECG signal and for each anomaly type.

In [ ]:
from plot_example_beats import plot_examples
plot_examples()

# Hardware

We will first run a software simulation of our SNN and then run the network directly on a DYNAP-SE neuromorphic processor. The device we are demonstrating here imposes a few restrictions on the network which are described below and which will also be considered in the software simulations.

## Core-wise parameters
Neuron and synapse parameters, such as time constants, firing thresholds and weights are set per core. The present processor consists of 16 cores of 256 neurons each. 

## Discrete weights
Synaptic weights are the same for each postsynaptic neuron on a core, resulting in ternary weights: positive (excitatory), negative (inhibitory), and zero (not connected). However, between each pair of neurons, multiple connections are possible, which effectively allows for integer weights.

## Connectivity
The number of presynaptic connections to each neuron is generally limited to 64.



# Input data

...

# Signal-to-spike encoding

The analog ECG signal is converted to trains of events through a sigma-delta encoding scheme. For every ECG lead there are two output channels, emitting spikes when the input signal increases ("up"-channel) or decreases ("down"-channel) by a specified amount.

The four resulting spike trains serve as input to the acutal network.

In [14]:
# - Spike encoding
spike_enc = FFUpDown(
    weights=NUM_ECG_LEADS,
    dt=DT_ECG,    
    thr_up=0.1,
    thr_down=0.1,
    multiplex_spikes=True,
    name="spike encoder"
)

# Network architecture


To detect the anomalies we will use a reservoir network consisting of three layers:

## Input expansion layer

This layer consists of 128 neurons, each of which with up to 64 presynaptic excitatory connections (or a positive integer weight up to 64) to one of the four input channels. This connection scheme ensures that the neurons respond differently to the input, therefore increasing the dimensionality of the signal. 

This is enhanced by the fact that the hardware neurons on the DYNAP-SE slightly vary in their individual characteristics, which will result in richer neuron dynamics.

In [2]:
%matplotlib widget

# - Input expansion layer
num_ch_in = 2 * NUM_ECG_LEADS
size_expand = 128
baseweight_expand = 5e-4

# weights_expand = np.zeros((num_ch_in, size_expand))
# num_input_conns = np.random.randint(1, MAX_FANIN + 1, size=size_expand)
# num_neur_per_ext = int(np.floor(size_expand / num_ch_in))
# for idx_ch_in in range(num_ch_in):
#     idcs_inp = slice(idx_ch_in * num_neur_per_ext, (idx_ch_in + 1) * num_neur_per_ext)
#     weights_expand[idx_ch_in, idcs_inp] = num_input_conns[idcs_inp]
    
weights_expand = np.load("weights_expand.npy")

plt.imshow(weights_expand, aspect="auto")
plt.title("Weights of dimensionality expansion layer", fontsize=18)

# Scale weights for software simulation
weights_expand_scaled = weights_expand * baseweight_expand

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# Load parameters from file (generated with gen_params.py)
kwargs_expand = dict(np.load("kwargs_expand.npz"))

# - Instantiate layer object
inp_expansion = FFIAFSpkInRefrTorch(
    weights=weights_expand_scaled,
    name="input_expansion_layer",
    **kwargs_expand
)

Layer `input_expansion_layer`: Using CPU as CUDA is not available.


## Reservoir layer

The reservoir layer consists of 512 excitatory and neuron 128 inhibitory neurons which are connected recurrently in a stochastic manner. Due to the recurrent connections, the network state at a given time does not only depend on the current input but also on previous network states, therefore implicitly encoding information about past inputs. This makes it possible to processs temporal relations in the input signal.

As a result, the state of each neuron in the reservoir is a function of the history of the input signal. If the functions ... by all neurons together are linearly independent (have high dimensionality) ... can combine them to approximate arbitrary functions. This linear combniation is done by the readout layer.

The separation into excitatory and inhibitory neurons is not strictly required but makes it easier to control neuron dynamics on the neuromorhpic processor.

In [11]:
## -- Reservoir layer
size_rec_exc = 512
size_inh = 128
size_reservoir = size_rec_exc + size_inh

# - Connections to excitatory layer
num_exp_rec = 16
num_rec = 32
num_inh = 16
# - Connections to inhibitory layer
num_rec_inh = 64

baseweight_exp_rec = 8e-5
baseweight_rec = 1.75e-4
baseweight_rec_inh = 8e-5
baseweight_inh = 1e-4

# # - Input connections to reservoir
# weights_res_in = np.zeros((size_expand, size_reservoir))

# # Connections only go to excitatory part or reservoir (first `size_rec_exc` neurons)
# presyn_conns_in = np.random.randint(size_expand, size=(size_rec_exc, num_exp_rec))
# for i_post, pre_neurs in enumerate(presyn_conns_in):
#     for i_pre in pre_neurs:
#         weights_res_in[i_pre, i_post] += 1

# # - Recurrent reservoir connections
# weights_rec = np.zeros((size_reservoir, size_reservoir))

# # Excitatory recurrent connections
# presyn_conns_exc = np.random.randint(size_rec_exc, size=(size_rec_exc, num_rec))
# for i_post, pre_neurs in enumerate(presyn_conns_exc):
#     for i_pre in pre_neurs:
#         weights_rec[i_pre, i_post] += 1

# # Connections from excitatory to inhibitory population
# presyn_conns_exc_inh = np.random.randint(size_rec_exc, size=(size_inh, num_rec_inh))
# for i_post, pre_neurs in enumerate(presyn_conns_exc_inh):
#     for i_pre in pre_neurs:
#         weights_rec[i_pre, i_post + size_rec_exc] += 1

# # Inhibitory connections
# presyn_conns_inh = np.random.randint(size_inh, size=(size_rec_exc, num_inh))
# for i_post, pre_neurs in enumerate(presyn_conns_inh):
#     for i_pre in pre_neurs:
#         weights_rec[i_pre + size_rec_exc, i_post] += 1
        
# # - For each neuron count number of presynaptic connections
# if (np.sum(weights_res_in, axis=0) + np.sum(weights_rec, axis=0) == MAX_FANIN).all():
#     print(f"All neurons have exactly {MAX_FANIN} presynaptic connections.")

weights_res_in = np.load("weights_res_in.npy")
weights_rec = np.load("weights_rec.npy")

In [5]:
%matplotlib widget
plt.imshow(weights_res_in, aspect="auto")
plt.title("Connections from expansion layer to reservoir")
plt.xlabel("Postsynaptic neurons")
plt.ylabel("Presynaptic neurons")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Presynaptic neurons')

In [6]:
%matplotlib widget
plt.imshow(weights_rec, aspect="auto")
plt.title("Recurrent reservoir connections")
plt.xlabel("Postsynaptic neurons")
plt.ylabel("Presynaptic neurons")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Presynaptic neurons')

In [8]:
# - Load reservoir parameters from file (generated with gen_params.py)
kwargs_reservoir = dict(np.load("kwargs_reservoir.npz"))

# - Instantiate reservoir layer object
reservoir = RecIAFSpkInNest(
    weights_in=weights_res_in,
    weights_rec=weights_rec,
    name="reservoir layer"
    **kwargs_reservoir
)

## Readout layer
- filter reservoir spikes 
- combine filtered spike trains
- use regression to train weights such that output is close to target (1 for anomaly, 0 for normal)

In [12]:
readout = FFExpSyn(
    weights=np.zeros((size_reservoir, NUM_ECG_CLASSES)),
    bias=0,
    dt=DT_ECG,
    tau_syn=0.175,
    name="readout layer"
)

In [ ]:
## Network

In [15]:
sw_net = Network(spike_enc, inp_expansion, reservoir, readout, dt=DT_ECG)

# Software Simulation

## Training
(just load pre-trained data, but show how training would work)

In [ ]:
# - Object to load ECG data
data_loader = ECGDataLoader()

# - Generator that yields batches of ECG data
batchsize_training = 40
num_beats = 100
regularize = 
batch_gen = data_loader.get_batch_generator(num_beats=num_beats, batchsize=batchsize_training)

for batch in batch_gen:
    output = sw_net.evolve(batch.input)
    readout.train_rr(
        batch.target,
        output["reservoir layer"],
        is_first=batch.is_first,
        is_last=batch.is_last,
        regularize=0.001,
    )

## Inference


# Hardware Implementation\
## Training

## Inference
